In [5]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [6]:
from model import MukkeBudeLSTM
from mapping import MusicMapping
import utils
import music21 as m21
import tensorflow as tf
import numpy as np
from pathlib import Path

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

[]


In [7]:
# Create mappings
mapping = MusicMapping.create()
mapping.save("mapping.txt")

In [9]:
paths = m21.corpus.getComposer('bach')
paths = paths[:1]
print(f"Found {len(paths)} songs in corpus.")

encoded_songs = utils.load_dataset_lstm(paths, 64, mapping)

print(f"Songs encoded: {len(paths)}")

Found 1 songs in corpus.
Removed 0 bad songs
Remaining songs: 1
Songs encoded: 1728


In [10]:
# Train model
model = MukkeBudeLSTM(mapping)
print(model)
model.train(encoded_songs, 1)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 298)]       0         
                                                                 
 lstm (LSTM)                 (None, 256)               568320    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 298)               76586     
                                                                 
Total params: 644,906
Trainable params: 644,906
Non-trainable params: 0
_________________________________________________________________
26/26 [==============================] - 5s 160ms/step - loss: 2.7139 - accuracy: 0.6286


In [15]:
# Create song
generated_song = model.generate("n72 _ _ _ _ _ n72 _ _ _ _ _ n72 _ n71 _")    
new_song = utils.decode_songs_old(generated_song)

1/1 [==============================] - 0s 20ms/step


In [16]:
path = Path("generated_song_bach_lstm.mid")
utils.write_midi(new_song, path)

In [ ]:
# Open song in MuseScore (Linux if installed)
!musescore4portable {path.absolute()}

# Train with new Dataformat from chinese dude

In [28]:
# Load songs
paths = m21.corpus.getComposer('bach')
paths = paths[:10]
print(f"Found {len(paths)} songs in corpus.")

encoded_songs = []
for path in paths:
    song = utils.read_single_from_corpus(path)
    encoded_song = utils.to_polyphonic_encoding(song, mapping)
    encoded_songs.extend(encoded_song)

Found 10 songs in corpus.


d:\Dokumente2\Python_Programme\Komposition-eines-Musikstuecks-mittels-Neuronaler-Netze\mukkeBude\utils.py:291: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Note").highestTime,
d:\Dokumente2\Python_Programme\Komposition-eines-Musikstuecks-mittels-Neuronaler-Netze\mukkeBude\utils.py:292: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Chord").highestTime,


In [29]:
# Train model
model = MukkeBudeLSTM(mapping)
print(model)
model.train(encoded_songs)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None, 298)]       0         
                                                                 
 lstm_3 (LSTM)               (None, 256)               568320    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 298)               76586     
                                                                 
Total params: 644,906
Trainable params: 644,906
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
113/113 [==============================] - 22s 175ms/step - loss: 3.0330 - accuracy: 0.2286
Epoch 2/50
113/113 [==============================] - 20s 180ms/step - loss: 2.318

In [32]:
# Create song
generated_song = model.generate("xxbos n67 d4 n62 d4 n58 d4 n43 d4 xxsep d4 n67 d4 n62 d4 n58 d4 n55 d4 xxsep d4 n69 d4 n62 d4 n57 d4 n54 d4 xxsep")    

new_song_ints = mapping.numericalize(generated_song)
new_song = utils.from_polyphonic_encoding(np.array(new_song_ints), mapping, bpm=100)

path = Path("generated_song_bach_lstm.mid")
utils.write_midi(new_song, path)


1/1 [==============================] - 0s 22ms/step


IndexError: index 132 is out of bounds for axis 2 with size 128